## This workbook is intended to help on calling IFS Cloud Work Order API to create a task and Release it automatically

Import libraries

In [1]:
import random
import requests
import json
from datetime import datetime
from datetime import timedelta

Function to read condifuration file for secret keys (check file ifscloudconfig.txt)

In [2]:
def read_ifs_cloud_config_file(file: str):
    with open(file) as f:
        lines = f.readlines()
        client_id = lines[0].split('=')[1].strip()
        client_secret = lines[1].split('=')[1].strip()
        user = lines[2].split('=')[1].strip()
        password = lines[3].split('=')[1].strip()
    return client_id, client_secret, user, password

## Authenticate the user a get an access token

In [4]:
# Authentication URL
ifs_cloud_instance = "ohot-d04.build.ifsdemoworld.com"
ifs_cloud_instance_id = "ohotd041"
auth_url = f"https://{ifs_cloud_instance}/auth/realms/{ifs_cloud_instance_id}/protocol/openid-connect/token"

#read credentials from file
client_id, client_secret, username, password = read_ifs_cloud_config_file("../config/real-ifscloudconfig.txt")

# Authentication payload (replace with your actual credentials)
auth_payload = {
    "grant_type": "password",
    "client_id": client_id,
    "client_secret": client_secret,
    "username": username,
    "password": password
}

# Headers for the authentication request
auth_headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

# Make the POST request to get the access token
auth_response = requests.post(auth_url, headers=auth_headers, data=auth_payload)

# Check the response
if auth_response.status_code == 200:
    access_token = auth_response.json().get("access_token")
    print("Access token retrieved successfully")
else:
    print("Failed to retrieve access token:", auth_response.status_code, auth_response.text)


Access token retrieved successfully


## Call the IFS Cloud WorkOrderServices.svc/ReceiveWorkOrder API to create a Work Order

In [8]:

# URL of the API endpoint
url = f"https://{ifs_cloud_instance}/int/ifsapplications/projection/v1/WorkOrderServices.svc/ReceiveWorkOrder"


# Headers for the request
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Authorization": f"Bearer {access_token}"
}

#Define EarlyStart and LatestFinish
now = datetime.now()
end_date = now + timedelta(days=30)
earliestStart = now.strftime("%Y-%m-%dT%H:%M:%SZ")
latestFinish = end_date.strftime("%Y-%m-%dT%H:%M:%SZ")

#Variables
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
resourceGroup = 1353

# Work Types generator
workTypes = ["10", "20", "30", "40", "50", "60"]
workType = random.choice(workTypes)

# Payload with the JSON structure
payload = {
    "ReceiveWo": {
        "WorkTaskArray": [
            {   
                "Site": "211",
                "OrganizationSite": "211",
                "OrganizationId": "211",
                "Description": "WorkTask created using WorkOrderServices API",
                "ExcludeFromScheduling": False,
                "WorkTypeId": workType,
                "EarliestStart":"2024-03-29T08:00:00Z",
                "LatestFinish":"2024-12-31T14:00:00Z",
                "Duration":2.5,
                "WorkTaskResourceArray": [
                    {   "DemandType": "PERSON",
                        "SourcingOption": "InternallySourced",
                        "PlannedHours": 2.5,
                        "PlannedQuantity": 1,
                        "ResourceGroupSeq": resourceGroup
                    }
                ],
                "WorkTaskAddressArray": [
                    {
                        "AddressId": "1",
                        "Address1":"Via della Pelliccia, 12",
                        "ZipCode": "00153",
                        "City": "Rome",
                        "State":"",
                        "CountryCode": "IT"
                    }
                ]
            }
        ],
        "RegDate": "2024-07-25T01:01:01Z",
        "ErrDescr": "Test API for Task with address and demand",
        "OrgCode": "211",
        "Contract": "211",
		"OriginatingSystemId": timestamp
    }
}

# Convert the payload to JSON format
data = json.dumps(payload)

# Make the POST request to create the work order
response = requests.post(url, headers=headers, data=data)

# Check the response
if response.status_code == 200:
    print("Work order created successfully:", response.json())
    newWorkOrder = response.json()['value']
else:
    print("Failed to create work order:", response.status_code, response.text)


Work order created successfully: {'@odata.context': 'https://ohot-d04.build.ifsdemoworld.com/int/ifsapplications/projection/v1/WorkOrderServices.svc/$metadata#Edm.Decimal', 'value': 17}


## Release the Work Order created

Before releasing a WO using the proper action it's needed to retrieve the e-tag from the Work Order to ensure object has not been changed meanwhile,
<b>Please Note</b> this is just an example but e-tag is not used because the Work Order will get automatically released when all Work Tasks underneath are released (and that's what we will do in the step following this)

In [6]:

#Get the e-tag
urlGetWorkOrder = f"https://{ifs_cloud_instance}/main/ifsapplications/projection/v1/ActiveWorkOrdersHandling.svc/ActiveSeparateSet?$filter=(WoNo eq {newWorkOrder})"

# Headers for the request
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Authorization": f"Bearer {access_token}"
}

# Make the GET request to get the e-tag of the work order
responseGetWorkOrder = requests.get(urlGetWorkOrder, headers=headers)
responseJson = json.loads(responseGetWorkOrder.text)

if responseGetWorkOrder.status_code == 200:
    # Extract the ETag from the response headers
    etag = responseJson['value'][0]['@odata.etag']
    print("ETag retrieved:", etag)
else:
    print("Failed to create work order:", responseGetWorkOrder.status_code, responseGetWorkOrder.text)


ETag retrieved: W/"Vy8iQUFBWkdHQUFpQUFGeGUwQUFBOjIi"


In [87]:

#Release the WO

# URL of the API endpoint
url = f"https://ohot-d04.build.ifsdemoworld.com/int/ifsapplications/projection/v1/WorkOrderServices.svc/ActiveWorkOrderSet(WoNo={newWorkOrder})/IfsApp.WorkOrderServices.ActiveSeparate_Release"


# Headers for the request
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Authorization": f"Bearer {access_token}",
    "If-Match": etag
}

payload = "{}"

# Make the POST request to create the work order
response = requests.post(url, headers=headers, data=payload)

# Check the response
if response.status_code == 204:
    print("Work order releases:", response.text)
else:
    print("Failed to release the work order:", response.status_code, response.text)

Work order releases: 


## Release all the Work Task underneath the Work Order (WO will be then automatically released)

In [9]:
#Release all tasks in the work order
urlGetAllTasksPerWO = f"https://{ifs_cloud_instance}/main/ifsapplications/projection/v1/WorkTasksHandling.svc/JtTaskSet?$filter=(WoNo eq {newWorkOrder})"

# Headers for the request
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
    "Authorization": f"Bearer {access_token}"
}

# Make the GET request to get the e-tag of the work order
responseGetTasksPerWorkOrder = requests.get(urlGetAllTasksPerWO, headers=headers)
responseJson = json.loads(responseGetTasksPerWorkOrder.text)

print("Info:", responseGetWorkOrder.status_code, responseGetTasksPerWorkOrder.text)

#Number of tasks to release
nbrTask = len(responseJson['value'])

#Loop through all the task
for taskIndex in range(nbrTask):
    taskId = responseJson['value'][taskIndex]['TaskSeq']

    #Get the etag for the task
    urlGetTask = f"https://{ifs_cloud_instance}/main/ifsapplications/projection/v1/WorkTasksHandling.svc/JtTaskSet(TaskSeq={taskId})"
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    responseGetTask = requests.get(urlGetTask, headers=headers)
    responseJsonTask = json.loads(responseGetTask.text)
    etagTask = responseJsonTask['@odata.etag']


    #Release the task
    url = f"https://{ifs_cloud_instance}/int/ifsapplications/projection/v1/WorkTaskServices.svc/WorkTaskSet(TaskSeq={taskId})/IfsApp.WorkTaskServices.JtTask_Release"
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {access_token}",
        "If-Match": etagTask
    }
    payload = "{}"
    response = requests.post(url, headers=headers, data=payload)

    # Check the response
    if response.status_code == 204:
        print("Work Task releases:", response.text)
    else:
        print("Failed to release the work task:", response.status_code, response.text)



Info: 200 {"@odata.context":"https://ohot-d04.build.ifsdemoworld.com/main/ifsapplications/projection/v1/WorkTasksHandling.svc/$metadata#JtTaskSet","value":[{"@odata.etag":"W/\"Vy8iQUFBWlpmQUFpQUFGdjloQUFKOjIi\"","luname":"JtTask","keyref":"TASK_SEQ=262^","Objstate":"NEW","Objgrants":null,"TaskSeq":262,"OrderNo":10,"WoNo":17,"Site":"211","Company":"27","OrganizationSite":"211","OrganizationId":"211","PriorityId":null,"WorkTypeId":"40","Description":"WorkTask created using WorkOrderServices API","LongDescription":null,"CreatedBy":"TISCNL","CreatedDate":"2024-07-26T11:40:44Z","PreparedBy":null,"ReportedBy":"TISCNL","ReportedDate":"2024-07-26T11:40:44Z","MpbLatestUpdate":null,"PlannedStart":null,"PlannedFinish":null,"Duration":2.5,"ActualStart":null,"ActualFinish":null,"EarliestStart":"2024-03-29T08:00:00Z","LatestStart":null,"LatestFinish":"2024-12-31T14:00:00Z","FixedStart":null,"SlaOrderNo":null,"SlaOrderLineNo":null,"ExcludeFromScheduling":false,"AdjustedDuration":false,"Remark":null,"